In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
import tika
from tika import parser
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')



In [2]:
base = ['data/Ml_', 'data/Bd_', 'data/Hd_', 'data/AS_']
data = []
num_data = []
for elem in range(0, len(base)):
    count = 0
    final = 10    
    for i in range(1, final):
        terminal = base[elem] + str(i) + '.pdf'
        data.append(terminal)
        count+=1
    
    num_data.append(count)



In [3]:
data

['data/Ml_1.pdf',
 'data/Ml_2.pdf',
 'data/Ml_3.pdf',
 'data/Ml_4.pdf',
 'data/Ml_5.pdf',
 'data/Ml_6.pdf',
 'data/Ml_7.pdf',
 'data/Ml_8.pdf',
 'data/Ml_9.pdf',
 'data/Bd_1.pdf',
 'data/Bd_2.pdf',
 'data/Bd_3.pdf',
 'data/Bd_4.pdf',
 'data/Bd_5.pdf',
 'data/Bd_6.pdf',
 'data/Bd_7.pdf',
 'data/Bd_8.pdf',
 'data/Bd_9.pdf',
 'data/Hd_1.pdf',
 'data/Hd_2.pdf',
 'data/Hd_3.pdf',
 'data/Hd_4.pdf',
 'data/Hd_5.pdf',
 'data/Hd_6.pdf',
 'data/Hd_7.pdf',
 'data/Hd_8.pdf',
 'data/Hd_9.pdf',
 'data/AS_1.pdf',
 'data/AS_2.pdf',
 'data/AS_3.pdf',
 'data/AS_4.pdf',
 'data/AS_5.pdf',
 'data/AS_6.pdf',
 'data/AS_7.pdf',
 'data/AS_8.pdf',
 'data/AS_9.pdf']

In [4]:
# data = ['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf']
biblioteca = []
tika.initVM()
for i in range(0, len(data)):
    
    parsed = parser.from_file(data[i])
    metadados = parsed["metadata"]
    texto = parsed["content"]
    biblioteca.append(texto)

In [5]:
print(len(biblioteca))
print(len(data))

36
36


In [6]:
# f = open ("artigo2.txt", "w")
# f.write (str(texto))
# f.close ()

In [7]:

# f = open ("artigo2.txt", "r")
# texto = f.read()

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
def split_pdf(string): 
    li = list(string.split(" ")) 
    return li

In [10]:
def first_clear(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    #remove links and mark
    data = [re.sub('(https:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(http:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(www.:\S+)|(@)|(¿)', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove spaces in begining and end
    data = [re.sub(' +', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    return data

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
def convert(s): 
    a = 0
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x + ' '
        a+=1
  
    # return string  
    return new, a

In [15]:

data_cleared = []
train_cleared = []

In [16]:
total_char = 0
for i in range(0, len(biblioteca)):
    texto = biblioteca[i]
    data = split_pdf(texto)
    data = first_clear(data)
    d, a = convert(data)
    
    total_char += a
    data_cleared.append(d)


In [17]:
print(len(data_cleared))
print(len(train_cleared))
print(total_char)

36
0
292303


In [18]:
tagged_data = []
for i, _d in enumerate(data_cleared):
    if i < num_data[0]:
        label = 'Ml'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    elif i < (num_data[0] + num_data[1]):
        label = 'Bd'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    elif i < (num_data[0] + num_data[1] + num_data[2]):
        label = 'Hd'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))
    else:
        label = 'As'+ '_' + str(i)
        tagged_data.append(TaggedDocument(words=word_tokenize(_d), tags=[label]))


In [26]:
# tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(data_cleared)]
# tagged_data[num_data[0]-1][1]

['Ml_8']

In [28]:
train_data = tagged_data[0:(num_data[0]-3)] + tagged_data[(num_data[0]):(num_data[0] + num_data[1]-3)] \
+ tagged_data[(num_data[0] + num_data[1]):(num_data[0] + num_data[1] + num_data[2]-3)] + \
tagged_data[(num_data[0] + num_data[1] + num_data[2]):-3]

In [29]:
test_data = tagged_data[num_data[0]-2:num_data[0]] + tagged_data[(num_data[0] + num_data[1]-2):num_data[0] + num_data[1]] \
+ tagged_data[(num_data[0] + num_data[1] + num_data[2]-2):(num_data[0] + num_data[1] + num_data[2])] +\
tagged_data[-2:]

In [30]:
len(test_data)
for i in range(0, len(test_data)):
    print(test_data[i][1])

['Ml_7']
['Ml_8']
['Bd_16']
['Bd_17']
['Hd_25']
['Hd_26']
['As_34']
['As_35']


In [31]:
max_epochs = 30
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(train_data)

for epoch in range(max_epochs):
#     print('iteration {0}'.format(epoch))
    model.train(train_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


In [32]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data

for i in range(0, len(test_data)):
    test = word_tokenize(str(test_data[i][0]))
    v1 = model.infer_vector(test)
    print(test_data[i][1])

    # to find most similar doc using tags
    similar_doc = model.docvecs.most_similar([v1])
    print(similar_doc)


['Ml_7']
[('Ml_1', 0.3849268853664398), ('Ml_5', 0.3821370601654053), ('As_32', 0.3779486119747162), ('Bd_14', 0.3774511218070984), ('Ml_0', 0.3771664500236511), ('Bd_9', 0.37615078687667847), ('Ml_2', 0.3696645498275757), ('Hd_19', 0.3638122081756592), ('Ml_3', 0.36351823806762695), ('Hd_23', 0.35977232456207275)]
['Ml_8']
[('Ml_5', 0.4376177191734314), ('Ml_1', 0.434550404548645), ('As_30', 0.4298199415206909), ('Hd_19', 0.4146783947944641), ('As_32', 0.41424503922462463), ('Ml_2', 0.4095132648944855), ('Bd_14', 0.40632909536361694), ('Bd_11', 0.4027211666107178), ('Ml_4', 0.3999013602733612), ('Bd_9', 0.3996428847312927)]
['Bd_16']
[('Ml_1', 0.3905556797981262), ('Hd_18', 0.39039361476898193), ('Bd_10', 0.3892677426338196), ('Hd_19', 0.37671351432800293), ('As_31', 0.3730244040489197), ('Ml_0', 0.36794954538345337), ('As_32', 0.36565977334976196), ('Ml_5', 0.36384570598602295), ('Ml_2', 0.356100857257843), ('Bd_9', 0.3545700013637543)]
['Bd_17']
[('Ml_1', 0.4538269340991974), ('Hd_1